In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

titanic_path = kagglehub.competition_download('titanic')

print('Data source import complete.')


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# My imports
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
from sklearn.preprocessing import MinMaxScaler

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Okay, I'll leave the first mandatory block already pre installed with kaggle as it was, and from now on, my contribution to the problem begins.

# Section 0 : Data acquisition, validation and preprocessing

In [ ]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
train_data.shape
#print(training_data.head())

In [ ]:
test_data.shape
#print(testing_data.head())

In [ ]:
train_data.isnull().sum();
test_data.isnull().sum();

There are a variety of formats in the data, so now I should turn it to a quantitative value in order for the model to be able to understand it. The non-numerical columns are Name, Sex, Ticket, Cabin and Embarked.

**a) The Name Column:**

For the name I believe that the title of each person could provide important information. I know the different possible titles thanks to a a little code that gets the different options independently from the output (as long as the format is kept the same)

In [ ]:
# Extract titles from the "Name" column
train_data['Name'] = train_data['Name'].str.extract(r',\s*([^\.]+)\.')

title_counts = train_data['Name'].value_counts()
print("Unique Titles with Counts:")
print(title_counts)

Now, comes the step to rework the values.

For the time being, let's propose the following change:
* There will be 5 categories: Mr, Mrs, Master, Miss and 'Special'
* It will go as simple as this:
  * Miss that will include Miss, Mlle, Mme and Ms
  * Mr that will include Mr and Sir and
  * Mrs that will include Mrs and Lady,
  * Master will only be assigned himself
  * The rest will all be in a 'Special' category

First let's design the function that will do the swapping according to our rules:

In [ ]:
# Define a function to map titles to the specified categories
def categorize_title(title):
    if title in ['Miss', 'Mlle', 'Mme', 'Ms']:
        return 'Miss'
    elif title in ['Mr', 'Sir']:
        return 'Mr'
    elif title in ['Mrs', 'Lady']:
        return 'Mrs'
    elif title == 'Master':
        return 'Master'
    else:
        return 'Special'

In [ ]:
train_data['Name'] = train_data['Name'].apply(categorize_title)

# Map title categories to numerical values
title_mapping = {'Mr': 1, 'Mrs': 2, 'Master': 3, 'Miss': 4, 'Special': 5}
train_data['Name'] = train_data['Name'].map(title_mapping)

In [ ]:
train_data.head()

**b) The Sex column**

Here the change is a whole lot more straightforward to apply, male will be 0 and female will be 1

In [ ]:
# Map title categories to numerical values
sex_mapping = {'male': 0, 'female': 1, '': 2}
train_data['Sex'] = train_data['Sex'].map(sex_mapping)

In [ ]:
train_data.head()

**c) The Age column**

In this case what happens is that 177 entries are missing a value, and the model causes problems whenever we lack data. We could implement more advanced solutions, but let's keep it simple for the time being

In [ ]:
# Fill missing Age values with the median
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median())

**d) The Ticket column**

Here come again the complications, because the ticket denomination is not as straight as the other parameters.

We could treat it the same way as with the title column, firstly obtaining the possible prefixes (being aware of possible hyphons)

In [ ]:
# Extract prefixes from the "Ticket" column
train_data['Ticket'] = (
    train_data['Ticket']
    .str.extract(r"^([^\d]*)")[0]  # Extract prefix
    .fillna('')                   # Handle missing prefixes
    .str.replace(r"[^\w\s]", "", regex=True)  # Remove punctuation
    .str.strip()                  # Strip extra whitespace
    .str.upper()                  # Converts all to higher case
)

ticket_counts = train_data['Ticket'].value_counts()
print("Unique Ticket prefixes with Counts:")
print(ticket_counts)

A quick internet search helps us to understand their meaning and their 'social class' affiliation
* **Wealthy First-Class Passengers:**
Often have prefixes like PC or SC.
* **Third-Class Emigrants:**
Typically have prefixes like A/5, STON/O, or CA.
* **Group Tickets:**
Shared prefixes might indicate families or groups traveling together.

In [ ]:
# Define a function to map ticket prefixes to the specified categories
def categorize_ticket(prefix):
    if prefix in ['PC', 'SCPARIS', 'WEP']:
        return 'First'
    elif prefix in ['SOC', 'SCA', 'CASOTON']:
        return 'Second'
    elif prefix in ['A', 'STONO', 'AS', 'CA', 'SOTONO', 'SOTONOQ', 'WC']:
        return 'Third'
    elif prefix in ['LINE' , 'FCC', 'FC']:
        return 'Crew'
    elif prefix == '':
        return 'Numbered'
    else:
        return 'Else'

In [ ]:
train_data['Ticket'] = train_data['Ticket'].apply(categorize_ticket)

# Map title categories to numerical values
ticket_mapping = {'First': 1, 'Second': 2, 'Third': 3, 'Crew': 4, 'Numbered': 5, 'Else': 6}
train_data['Ticket'] = train_data['Ticket'].map(ticket_mapping)

In [ ]:
train_data.head()

**e) The Cabin column**

Similar to the ticket, we'll need some regular expression to filter the different values and then assign them to a Descriptor.

In [ ]:
# Extract prefixes from the "Cabin" column
train_data['Cabin'] = train_data['Cabin'].str.extract(r"^([A-Za-z])")

cabin_counts = train_data['Cabin'].value_counts()
print("Unique Cabin letters with Counts:")
print(cabin_counts)

In [ ]:
# Define a function to map cabin letters to the specified categories
def categorize_cabin(cabin):
    if cabin in ['A', 'B', 'C']:
        return 'First'
    elif cabin in ['D']:
        return 'Second'
    elif cabin in ['E', 'F']:
        return 'Third'
    elif cabin == '':
        return 'None'
    else:
        return 'Else'

In [ ]:
train_data['Cabin'] = train_data['Cabin'].apply(categorize_cabin)

# Map title categories to numerical values
cabin_mapping = {'First': 1, 'Second': 2, 'Third': 3, 'None': 4, 'Else': 5}
train_data['Cabin'] = train_data['Cabin'].map(cabin_mapping)

In [ ]:
train_data.head()

**f) The Embark column**

A lot easier to rework, just differentiate between the 3 possible options

In [ ]:
embark_mapping = {'C': 1, 'Q': 2, 'S': 3}
train_data['Embarked'] = train_data['Embarked'].map(embark_mapping)
train_data = train_data.dropna(subset=['Embarked'])

In [ ]:
train_data.head()

# Section 0.5 Normalization

To avoid that certain values during the calculus get overblown, let's import an scaler to normalize the whole dataset

In [ ]:
# Initialize the scaler
scaler = MinMaxScaler()

# Normalize predictors
predictors = pd.DataFrame(scaler.fit_transform(train_data.drop(columns=['Survived'])), columns=train_data.drop(columns=['Survived']).columns)
target = train_data['Survived']

n_cols = predictors.shape[1]

Let's check if everything is up to par and we can go on with the model

In [ ]:
print(predictors.isnull().sum())
print(target.isnull().sum())

# Section 1 : Lineal Regression

In [ ]:
# define regression model
def regression_model(units):
    # create model
    model = Sequential([
        Input(shape=(n_cols,)),
        Dense(units,activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # compile model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipvalue=1.0)
    model.compile(optimizer=optimizer, loss='binary_crossentropy')
    return model

In [ ]:
#build the model
model_r = regression_model(100)

In [ ]:
history_r = model_r.fit(
    predictors, target,         # Training data
    validation_split=0.15,       # Use 15% of the training data for validation
    epochs=50,                  # Number of training epochs
    batch_size=32              # Number of samples per gradient update
    #verbose=0                   # Display progress
)